In [47]:
import sqlite3 
import pandas as pd 
import requests
import json  
from datetime import datetime
import secrets

def connect_create_table(name_db='db.db'):
    conn = sqlite3.connect(name_db)  # You can create a new database by changing the name within the quotes
    c = conn.cursor() # The database will be saved in the location where your 'py' file is saved

    # Create table - CLIENTS
    c.execute('''CREATE TABLE if not exists POSITIONS
                 ([id] INTEGER PRIMARY KEY,
                 [date] DATETIME,
                 [encryptedUid] TEXT,
                 [nick] TEXT,
                 [symbol] TEXT,
                 [entryPrice] FLOAT,
                 [markPrice] FLOAT,
                 [pnl] FLOAT,
                 [roe] FLOAT,
                 [amount] FLOAT,
                 [updateTimeStamp] TEXT,
                 [request_id] TEXT
                 )''')
    conn.commit()

connect_create_table()

In [48]:
def insert_in_table(trader_info, request_id, vector):
    encryptedUid = trader_info[0]
    nick         = trader_info[1]
    
    try:
        sqliteConnection = sqlite3.connect('db.db')
        cursor = sqliteConnection.cursor()
        
        date        = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        symbol      = vector[0]
        entryPrice  = vector[1]
        markPrice   = vector[2]
        pnl         = vector[3]
        row         = vector[4]
        amount      = vector[5]
        updateTimestamp = vector[6]

        # Need to add more info of student (such as the txns)
        sqlite_insert_with_param = """INSERT INTO POSITIONS 
                          (date, encryptedUid, nick, symbol, entryPrice, markPrice, pnl, roe, amount, updateTimeStamp, request_id) 
                          VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?,?);"""

        data_tuple = (date, encryptedUid, nick, symbol, entryPrice, markPrice, pnl, row, amount, updateTimestamp,request_id) 
        cursor.execute(sqlite_insert_with_param, data_tuple)
        sqliteConnection.commit()

        cursor.close()

    except sqlite3.Error as error:
        print("Failed to insert Python variable into sqlite table", error)
    finally:
        if sqliteConnection:
            sqliteConnection.close()

In [49]:
def do_query(db="db.db", query="SELECT * FROM POSITIONS"):    
    con = sqlite3.connect(db)
    df = pd.read_sql_query(query, con)
    
    return df

In [50]:
def get_positions(trader):
    endpoint = "https://www.binance.com/bapi/futures/v1/public/future/leaderboard/getOtherPosition"
    params   =  {"encryptedUid":trader,  "tradeType":"PERPETUAL"}
    headers  =  {"content-type":"application/json;charset=UTF-8"}
    response  = requests.post(endpoint, json=params, headers=headers) #Add proxies = proxyDict # Attention to JSON = PARAMS
    result = response.json()["data"]["otherPositionRetList"]
    return result

In [51]:
traders  = ["CCF3E0CB0AAD54D9D6B4CEC5E3E741D2", "49A7275656A7ABF56830126ACC619FEB", "A532C4316C00206168F795EDFBB3E164", "D3AFE978B3F0CD58489BC27B35906769",
            "FB23E1A8B7E2944FAAEC6219BBDF8243", "F318AE1B1BB3AEF4EDBD36E5AE250CD1", "2F9D01E80429F97670415A5A5DDD9405"]
names =    {"CCF3E0CB0AAD54D9D6B4CEC5E3E741D2":"TraderT", "49A7275656A7ABF56830126ACC619FEB":"ADnefina", "A532C4316C00206168F795EDFBB3E164":"ClickHereNow", "D3AFE978B3F0CD58489BC27B35906769":"CheerWindCard",
           "FB23E1A8B7E2944FAAEC6219BBDF8243":"BananaMan1", "F318AE1B1BB3AEF4EDBD36E5AE250CD1":"BigPiePlayer", "2F9D01E80429F97670415A5A5DDD9405":"CryptoPinoy"}

for trader in traders:
    keys   = ["symbol", "entryPrice", "markPrice", "pnl", "roe", "amount", "updateTimeStamp"]
    trader_info = [trader, names[trader]]
    request_id  = secrets.token_urlsafe(100)
    positions   = get_positions(trader)

    print(names[trader])
    
    # Check if new position is different:
        # Get last update from trader. Read the request_id
        # Get all the positions with that request_id
        # Compare the previous position with the new one, pair by pair
        # Calculate the difference
        # Format the difference
        # Send telegram message    

    # Save new positions in database
    if positions:
        for i in range(len(positions)):
            vector = [positions[i].get(key) for key in keys]
            insert_in_table(trader_info, request_id, vector)

TraderT
ADnefina
ClickHereNow
CheerWindCard
BananaMan1
BigPiePlayer
CryptoPinoy


In [52]:
do_query(query= "SELECT * FROM POSITIONS WHERE nick = 'TraderT'")

,id,date,encryptedUid,nick,symbol,entryPrice,markPrice,pnl,roe,amount,updateTimeStamp,request_id
0,1,2021-09-27 19:39:52,CCF3E0CB0AAD54D9D6B4CEC5E3E741D2,TraderT,SRMUSDT,7.750814,7.690664,-1956.219765,-0.039106,32522.000,1632761040868,EGzg78D5CTH-35hmzKJbE7fK51T-OdlheC-rydUBELYSD4...
1,2,2021-09-27 19:39:52,CCF3E0CB0AAD54D9D6B4CEC5E3E741D2,TraderT,BCHUSDT,499.543844,497.864829,-1369.873079,-0.033724,815.879,1632761544649,K33naufpqSLcfnJUXDxpgoon00RYkaKckJjZAJwSbygNju...
2,3,2021-09-27 19:39:52,CCF3E0CB0AAD54D9D6B4CEC5E3E741D2,TraderT,FILUSDT,60.845098,61.331445,7825.267194,0.079298,16089.900,1632763760145,p5iW_2QWnpxMPqpfEMlBciyjmxeRY28scSp5KQc22dLW4W...
3,4,2021-09-27 19:39:52,CCF3E0CB0AAD54D9D6B4CEC5E3E741D2,TraderT,SOLUSDT,141.508246,142.443303,6619.268503,0.065644,7079.000,1632762092618,3SWf8KCIsB4JRTKTU6k9E241ujgzBDukTLYa8TwrngsXbz...
4,5,2021-09-27 19:39:52,CCF3E0CB0AAD54D9D6B4CEC5E3E741D2,TraderT,OMGUSDT,8.970630,8.988320,199.007600,0.019680,11250.200,1632762169817,KD7tz3b7Cx8YypphZlsZU8WEI2CjOGDmN37MS7KExfbNwI...
5,6,2021-09-27 19:39:52,CCF3E0CB0AAD54D9D6B4CEC5E3E741D2,TraderT,FTMUSDT,1.292972,1.299078,1934.626361,0.023501,316848.000,1632761048946,bXbNRp0p5gfovrgE-PmTJFn7r2S7ZECflDKLzTpL2kiiFI...
6,7,2021-09-27 19:39:52,CCF3E0CB0AAD54D9D6B4CEC5E3E741D2,TraderT,DYDXUSDT,16.538537,19.299189,71390.181293,0.715225,25859.900,1632761227084,_1cY8v1OeMsUomHqUzG1-_d8r7qIJumKqdFSx55jxl3PZY...
7,37,2021-09-27 19:40:03,CCF3E0CB0AAD54D9D6B4CEC5E3E741D2,TraderT,SRMUSDT,7.750814,7.691544,-1927.598778,-0.038530,32522.000,1632761040868,H_kXomKQW_YJ7QzsA7mBOMYeENOHUqVC9OKhMU4haUvAsI...
8,38,2021-09-27 19:40:03,CCF3E0CB0AAD54D9D6B4CEC5E3E741D2,TraderT,BCHUSDT,499.543844,497.995768,-1263.043213,-0.031086,815.879,1632761544649,3zOGppL-M0qOhd0QkAxRzSZTho122gY3jim18VPygsZQnA...
9,39,2021-09-27 19:40:03,CCF3E0CB0AAD54D9D6B4CEC5E3E741D2,TraderT,FILUSDT,60.845098,61.347480,8083.283060,0.081891,16089.900,1632763760145,xfpknl3BomyqjAFP_HEi_8YcYVc351pBYlz1xEAewWsahT...
